# Techniques to profile and optimise python code

A common complaint about python is that "it's not as fast as C/C++/fortran/...". For example this recent article from Simon Portegies Zwart (https://arxiv.org/pdf/2009.11295.pdf) claims that one should not use python because it is inefficient, and therefore more environmentally harmful.

However, this statement was demonstrated to be untrue by Pierre Augier (see links from https://physicsworld.com/a/the-huge-carbon-footprint-of-large-scale-computing/). Augier argues that while python will be slower than other languages if one does not consider optimization (often not needed if you just want to test something, or do something quickly), for large-scale simulations it can be made to be faster than the implementations of Portegies Zwart's code in C/C++/fortran (https://github.com/paugier/nbabel).

So how do we do this? It is normally possible to write python code that is just as fast as code written in a "fast" (compiled) language such as C. After all, python is written in C and contains a wealth of libraries, such as numpy, that actually use compiled C (and even fortran) code in some of the most time-sensitive places.

I think its more accurate to say that its easier to write "slow" code in python, because it's often hard to see why some things are slow. To write code that *is* as fast as C does require a bit of understanding on how C actually works, but in most cases you can write code that is more than fast enough using the techniques shown here. In this lecture we will try to

* Highlight some of the places where python code is often slow and demonstrate ways to speed it up
* Demonstrate tools to "profile" python code, specifically to identify the functions, and lines, where the code is slowest

As a supplement to this, there are occasionally cases where python doesn't cut it in terms of speed. With the availability of numpy, *such cases are rare*. In such instances you can turn to "Cython" which is basically a hybrid of python and C/C++. We will cover Cython at the end of this course. Note that large parts of scipy are actually written in Cython, and while it is still nowhere near as well used as Python itself, it is increasing in popularity.

## Before we start

Before we start we will need to use a couple of utilities to demonstrate some of the profiling techniques. The cell below demonstrates how to do this in sciserver or Google Colab. **If using Sciserver (or Google colab) just run the cell below**.

**If not using sciserver or colab**: This might not be the right way to install these utilities. If you are using Anaconda (or Minconda or any other conda installed python) you will probably want to use the `conda` command shown below and commented out *instead*. This is normally run in a terminal window. I can try to help you install these on your own machine, but I do not know your particular setup, and do not use Windows.

In [ ]:
import sys
!{sys.executable} -m pip install line_profiler
!{sys.executable} -m pip install memory_profiler

# conda install line_profiler
# conda install memory_profiler

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 662.2/662.2 KB 10.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Load the stuff we imported above
%load_ext line_profiler
%load_ext memory_profiler

## An example

Here's an example code which integrates

$cos(x) \times \frac{1}{x} $

from $x = 1$ to $x=1000$.

We will do this by using the simple rectangular method for numerically integrating. https://en.wikipedia.org/wiki/Riemann_sum .... I know you all know the theory behind this, and how to code up such an example, as I taught it to you all in MATLAB last year!!

In [ ]:
# NOTE: We write this as a set of functions. Functions are better to isolate different parts of
#       the code and to be able to check each component individually. Using a class and class methods
#       to do this is also fine, and we mix both here to show this ... Might not be the most aesthetic
#       way of doing this, though! However, this does serve as a reminder: Don't write long blocks of code
#       split up into functions!
import numpy
import math

def compute_cosx(tseries):
    """
    Computes cos(t) for all values in tseries
    """
    cosx = numpy.zeros(len(tseries))
    for idx, tval in enumerate(tseries):
        cosx[idx] = math.cos(tseries[idx])
    return cosx

def compute_invx(tseries):
    """
    Computes 1/x for all values in tseries
    """
    invx = numpy.zeros(len(tseries))
    for idx, tval in enumerate(tseries):
        invx[idx] = 1 / tseries[idx]
    return invx

def compute_seriesproduct(series1, series2):
    """
    Multiplies each element in series1 with the corresponding element in series2.
    This returns an array of the multiplied elements.
    """
    # Ensure the two arrays are the same length
    assert(len(series1)==len(series2))
    seriessum = numpy.zeros(len(series1))
    for idx in range(len(series1)):
        seriessum[idx] = series1[idx] * series2[idx]
    return seriessum

def compute_seriessum(series):
    """
    Computes the sum of all values in series
    """
    sumvals = 0
    for idx in range(len(series)):
        sumvals = sumvals + series[idx]
    return sumvals


class Integrator():
    def generate_integral(self):
        """
        Integral function goes here
        """
        cosx = compute_cosx(self.tseries)
        invx = compute_invx(self.tseries)
        prod = compute_seriesproduct(cosx, invx)
        summed_prod = compute_seriessum(prod)
        return summed_prod * self.delta_t


    def __init__(self, tmin, tmax, delta_t):
        """
        Initializes the class and timeseries
        """
        self.tmin = tmin
        self.tmax = tmax
        self.delta_t = delta_t
        tseries = numpy.arange(self.tmin, self.tmax, self.delta_t)
        # We shift tseries by delta_t / 2 to ensure that we are using the midpoint rule (see wikipedia page)
        tseries = tseries + self.delta_t / 2.
        self.tseries = tseries


def main_function():
    intgr = Integrator(1, 10000, 1./300.)
    return intgr.generate_integral()

print (main_function())

-0.33743511454087033


## Starting to understand the code

Our first step in understanding the code is to time it. This can be done in the following way:

In [ ]:
%timeit main_function()

4.38 s ± 257 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Timeit will run the code a number of times and take an average. How many times it runs the code depends on how long the code takes (though you can override these values). Here we can see that our code took approximately a second to run. If running this only once then this is of course trivial.

**Important rule of optimising: Don't waste time optimising a block of code, unless it is slowing down your work ... If others are using your code, you must think about how they might use your code though, might it be a problem in the future? There are certainly some things in the code above that are unnecessarily slow, which can be made faster just by writing the code better in the first place**

Let's assume though that we might need to run this code thousands of times (or hundreds of thousands of times). In that case let's see what we can do to make it faster.

## Profiling code

Python and Jupyter notebooks have some neat tools for profiling code. Profiling means measuring how long the code takes to run individual blocks of code. Most profilers measure the fraction of time spent inside each individual *function*. Therefore splitting your code up into a number of different functions can help both in terms of making it easier to read and understand the code, but also in terms of understanding any bottlenecks.

Below we run a built-in profiler on our code above:

In [ ]:
%prun -l 10 -q -T prun0 main_function()

print(open('prun0', 'r').read())

 
*** Profile printout saved to text file 'prun0'. 
         2999721 function calls in 4.832 seconds

   Ordered by: internal time
   List reduced from 14 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    1.419    1.419    1.419    1.419 <ipython-input-2-9645487cbb32>:16(compute_invx)
        1    1.296    1.296    1.579    1.579 <ipython-input-2-9645487cbb32>:7(compute_cosx)
        1    1.062    1.062    1.062    1.062 <ipython-input-2-9645487cbb32>:25(compute_seriesproduct)
        1    0.754    0.754    0.754    0.754 <ipython-input-2-9645487cbb32>:37(compute_seriessum)
  2999700    0.281    0.000    0.281    0.000 {built-in method math.cos}
        1    0.009    0.009    0.009    0.009 {built-in method numpy.arange}
        1    0.004    0.004    0.013    0.013 <ipython-input-2-9645487cbb32>:59(__init__)
        1    0.004    0.004    4.832    4.832 <ipython-input-2-9645487cbb32>:72(main_function)
        3    0.002

The `tottime` entry is the amount of time spent within each function. We can see that the majority of time is spent computing `cos(x)`, `inv(x)` and doing the series product and sum.

To introduce all of our profiling tools in one place, let's also look at line-by-line and memory profiling. We can use the following to run the code to produce line-by-line profiling information

In [ ]:
timeseries = numpy.arange(1.0,1000.,0.01)
#compute_cosx(timeseries)

%lprun -T lprof0 -f main_function main_function()

print(open('lprof0', 'r').read())

# And we can also profile the sub-functions, such as compute_cosx

%lprun -T lprof0 -f compute_cosx compute_cosx(timeseries)

print(open('lprof0', 'r').read())

UsageError: Line magic function `%lprun` not found.


Finally, although we will not use it in this class, we can also profile the *memory usage* of a function in the same way. Unfortunately this only works for functions written in an external file, so we need to dump our code to a file and then run it from the file. Here's an example of that:

In [ ]:
%%file mprun_demo.py
import numpy
def invx_demo(tseries):
    """
    Computes 1/x for all values in tseries
    """
    invx = 1 / tseries
    return invx


Writing mprun_demo.py


In [ ]:
from mprun_demo import invx_demo
timeseries = numpy.arange(1.0,10000.,0.01)

%mprun -f invx_demo invx_demo(timeseries)


PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/usr/local/lib/python3.9/dist-packages/memory_profiler.py", line 847, in enable
    sys.settrace(self.trace_memory_usage)


PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/usr/local/lib/python3.9/dist-packages/memory_profiler.py", line 850, in disable
    sys.settrace(self._original_trace_function)



## Using these tools and information to speed things up

Okay, now that we've understood the tools available to us. Let's try to see if we can't improve things. The first place where the code is slow is in the `compute_cosx` function. Here's a major rule in python optimization:

* Avoid for loops wherever possible

In this case rather than using a python for loop to compute `cos(x)` at every index, let's use numpy to compute it at all indexes in one call. Yes, internally it will still need to loop over all values of `x` at some point and compute `cos(x)` for each point, but this will happen deep in some compiled numpy routine. In short

* Use numpy routines on vectors to avoid for loops where possible.

So we can replace our `compute_cosx` function with:

In [ ]:
def compute_cosx(tseries):
    """
    Computes cos(t) for all values in tseries
    """
    return numpy.cos(tseries)


Note that running this *after* the block above just replaces this function, so we can just run our profiler again:

In [ ]:
%prun -l 10 -q -T prun0 main_function()

print(open('prun0', 'r').read())

 
*** Profile printout saved to text file 'prun0'. 
         19 function calls in 3.131 seconds

   Ordered by: internal time
   List reduced from 13 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    1.444    1.444    1.444    1.444 <ipython-input-2-9645487cbb32>:16(compute_invx)
        1    1.068    1.068    1.068    1.068 <ipython-input-2-9645487cbb32>:25(compute_seriesproduct)
        1    0.557    0.557    0.557    0.557 <ipython-input-2-9645487cbb32>:37(compute_seriessum)
        1    0.047    0.047    0.047    0.047 <ipython-input-10-40c68ad4621a>:1(compute_cosx)
        1    0.007    0.007    0.007    0.007 {built-in method numpy.arange}
        1    0.005    0.005    0.012    0.012 <ipython-input-2-9645487cbb32>:59(__init__)
        1    0.003    0.003    3.131    3.131 <ipython-input-2-9645487cbb32>:72(main_function)
        2    0.000    0.000    0.000    0.000 {built-in method numpy.zeros}
        1    0.000 

Now we can see that quite a bit of time is being spent in the remaining 3 `compute_x` functions. Let's try optimizing these as well by replacing the for loops with numpy vectorized calls

In [ ]:
def compute_invx(tseries):
    """
    Computes 1/x for all values in tseries
    """
    return 1. / tseries

def compute_seriesproduct(series1, series2):
    """
    Multiplies each element in series1 with the corresponding element in series2.
    This returns an array of the multiplied elements.
    """
    # Ensure the two arrays are the same length
    assert(len(series1)==len(series2))
    return series1 * series2

def compute_seriessum(series):
    """
    Computes the sum of all values in series
    """
    return series.sum()

%prun -l 10 -q -T prun0 main_function()

print(open('prun0', 'r').read())


 
*** Profile printout saved to text file 'prun0'. 
         16 function calls in 0.110 seconds

   Ordered by: internal time
   List reduced from 15 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.072    0.072    0.072    0.072 <ipython-input-10-40c68ad4621a>:1(compute_cosx)
        1    0.013    0.013    0.013    0.013 {built-in method numpy.arange}
        1    0.010    0.010    0.010    0.010 <ipython-input-14-e2555c7ade69>:1(compute_invx)
        1    0.008    0.008    0.020    0.020 <ipython-input-2-9645487cbb32>:59(__init__)
        1    0.005    0.005    0.005    0.005 <ipython-input-14-e2555c7ade69>:7(compute_seriesproduct)
        1    0.002    0.002    0.002    0.002 {method 'reduce' of 'numpy.ufunc' objects}
        1    0.000    0.000    0.109    0.109 <ipython-input-2-9645487cbb32>:72(main_function)
        1    0.000    0.000    0.110    0.110 <string>:1(<module>)
        1    0.000    0.000    0.088  

The code is now limited be a vectorized computation of cos(x). Not easy to make that much faster! Now with these optimizations let's see how fast our code runs

In [ ]:
%timeit main_function()

59.1 ms ± 887 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


The code now runs *2 orders of magnitude* quicker. For loops over a large number of values can be very inefficient!

## Exercise 1

The following cells contain examples of python code that are written using explicit for loops. Time these functions (using `timeit`) and rewrite the for loop using numpy calls. After rewriting, time the functions again.

### EXERCISE 1.1

Here also investigate the time taken to compute these two functions. Which is faster?
Why do you think this is?
Numpy is really inefficient when running on scalar values.


In [ ]:
import numpy, math

def compute_sin_tseries():
    tseries = numpy.arange(1, 10000, 1./100.)
    sin_tseries = numpy.zeros(len(tseries))
    for i in range(len(timeseries)):
        sin_tseries[i] = numpy.sin(timeseries[i])
    return sin_tseries

def compute_sin_tseries2():
    tseries = numpy.arange(1, 10000, 1./100.)
    sin_tseries = numpy.zeros(len(tseries))
    for i in range(len(timeseries)):
        sin_tseries[i] = math.sin(timeseries[i])
    return sin_tseries

In [ ]:
# Solution to exercise 1.1 goes in here
def compute_optimized_sin_tseries():

In [ ]:
# Time your solution, and the original code, here


In [ ]:
# Then run this cell to verify it actually works
if numpy.allclose(compute_sin_tseries(), compute_optimized_sin_tseries()):
    print("Well done, this all looks good!")
else:
    print("Your optimized code doesn't seem to give the right output!")


### EXERCISE 1.2

In [ ]:
import math, numpy

def compute_exp_tseries():
    tseries = numpy.arange(1, 100, 1./10000.)
    exp_tseries = numpy.zeros(len(tseries))
    for i in range(len(tseries)):
        exp_tseries[i] = math.e ** tseries[i]
    return exp_tseries

In [ ]:
# Solution to exercise 1.2 goes in here
def compute_optimised_exp_tseries():

In [ ]:
# Then time your code (and the original) here

In [ ]:
# And test your code by running this cell
if numpy.allclose(compute_exp_tseries(), compute_optimised_exp_tseries()):
    print("Well done, this all looks good!")
else:
    print("Your optimized code doesn't seem to give the right output!")


### EXERCISE 1.3

Note that there are two for loops here. It is possible to collapse into one
single vectorized call, but simply removing one of the for loops will make the code both readable
and reasonably optimized. Feel free to try removing both loops, but it is *not at all* trivial, and not
really any faster than just removing the inner loop.

In [ ]:
import numpy as np

def sum_2d_array_to_1d(input_data):
    """
    This function takes as input a 2-dimensional array, for example:
    [[0,1,2,3],
     [2,2,2,2],
     [1,1,1,1],
     [10,11,12,13]]

    It should sum over each of the *rows* in turn and return the sum of each as a new array

    [6, 8, 4, 46]
    """
    output = np.zeros(len(input_data), dtype=float)
    for i in range(len(input_data)):
        # Looping over each row
        current_sum = 0
        for j in range(len(input_data[i])):
            # Looping over each element in each row, e.g. 10->11->12->13 if the bottom row
            current_sum += input_data[i][j]
        output[i] = current_sum
    return output

# Short example to test that it works
input_data = numpy.array([[0,1,2,3],
                          [2,2,2,2],
                          [1,1,1,1],
                          [10,11,12,13]])

print(sum_2d_array_to_1d(input_data))

# *This* is the large example to use timeit on and to make fast to run
input_data = numpy.random.random(size=[2000,2000])
print(sum_2d_array_to_1d(input_data))

[ 6.  8.  4. 46.]
[1005.97971962 1010.39164807 1003.76167116 ...  989.2619583  1002.88408881
  988.24284929]


In [ ]:
# Solution to exercise 1.3 goes in here
def sum_optimised_2d_array_to_1d(input_data):

In [ ]:
# Time your new function here

In [ ]:
# Then verify your code works here
input_data = numpy.random.random(size=[2000,2000])
if numpy.allclose(sum_2d_array_to_1d(input_data), sum_optimised_2d_array_to_1d(input_data)):
    print("Well done, this all looks good!")
else:
    print("Your optimized code doesn't seem to give the right output!")


### EXERCISE 1.4

This example performs a cross-correlation (This is something that you are going to see on Thursday. It will
be **very, very important** for the coursework??. The unoptimized
code below would work .... I just think it would take about 2 hours *per run* on the coursework data.


In [ ]:
def compute_cross_correlation(signal, data):
    cross_correlation = []
    for i in range(len(data) - len(signal)):
        curr_cross_corr = 0
        for j in range(len(signal)):
            curr_cross_corr += signal[j] * data[i+j]
        cross_correlation.append(curr_cross_corr)
    return cross_correlation

# An example of calling this
signal = numpy.random.random(1024)
data = numpy.random.random(1024*10)
compute_cross_correlation(signal, data)

In [ ]:
# Solution to exercise 1.4 goes in here .... I'm probably not going to release this in the solutions notebook.
def compute_optimised_cross_correlation(signal,data):

In [ ]:
# Time your new function here

In [ ]:
# Then verify your code works here
signal = numpy.random.random(1024)
data = numpy.random.random(1024*10)
if numpy.allclose(compute_cross_correlation(signal, data), sum_optimised_2d_array_to_1d(input_data)):
    print("Well done, this all looks good!")
else:
    print("Your optimized code doesn't seem to give the right output!")


## Exercise 2 - Square digits (again)

In the debugging lecture last year we showed an example of a [not working] code to do the following:

* Consider a sequence of numbers a0, a1, ..., an, in which an element is equal to the sum of squared digits of the previous element. The sequence ends once an element that has already been in the sequence appears again. So that an = a0. Given the first element a0, find the length of the sequence.


So for example if a0 = 16. `1**2 + 6**2 = 37` -> `3**2 + 7**2 = 58` -> `5**2 + 8**2 = 89` -> `8**2 + 9**2 = 145` -> `1**2 + 4**2 + 5**2 = 42` -> `4**2 + 2**2 = 20` -> `2**2 + 0**2 = 4` -> `4**2 = 16` We've already seen 16 so we stop here. The sequence is `[16,37,58,89,145,42,20,4,16]` which has a length of 9, return 9.

The code that we used had some optimisations, but was not particularly clear. To try to understand it better let's write this out in a code that approaches the problem in a different way, with comments etc. to try to make it clearer what is going on

In [ ]:
def square_digits(input_number):

    # Initialize by setting the list of outputs equal to the input
    output_list = [input_number]
    # And setting the last_number variable to the input
    last_number = input_number

    # This is basically a for-loop that will only exit when we explicitly say "exit"
    while 1:
        # Step one: We must identify the digits of last_number
        # Cast to a string
        last_number = str(last_number)
        # And then convert to a list of integers. We do this using a list comprehension, which is powerful, but not fast
        digits = [int(digit) for digit in last_number]
        # So if last_number is 49120 then digits = [4,9,1,2,0]

        # We can then sum the digits squared using another list comprehension
        # digits = [4,9,1,2,0] -> 16 + 81 + 1 + 4 = 102
        digit_squared_sum = sum([digit*digit for digit in digits])

        # Is this value already in the list?
        if digit_squared_sum in output_list:
            # Add this value and then exist
            output_list.append(digit_squared_sum)
            break
        else:
            # Else add the value and then continue
            output_list.append(digit_squared_sum)
            last_number = digit_squared_sum

    return len(output_list)

print("square_digits(103) gives:",square_digits(103), "\n Should be 4")
print()
print("square_digits(612) gives:",square_digits(612), "\n Should be 16")
print()


square_digits(103) gives: 4 
 Should be 4

square_digits(612) gives: 16 
 Should be 16



In [ ]:
# Let's try it with the following
very_long_integer = 2**100
square_digits(very_long_integer)

13

Using the profiling tools demonstrated above:

* Use timeit to calculate how long the function takes to run with the `very_long_integer`
* Use lprun to determine how long the code spends at each line

In [ ]:
# Run timeit and lprun here


Here is the optimized code that we used last year (with the bug fixed). As before run timeit and lprof on this code:

In [ ]:
def square_digits_withoutstr(input_number):

    cur = input_number
    was = set()

    while not (cur in was):
        was.add(cur)
        nxt = 0
        while cur > 0:
            nxt += (cur % 10) * (cur % 10)
            cur //= 10
        cur = nxt

    return len(was) + 1


In [ ]:
# Run timeit and lprun here


Now repeat the process using:
```very_long_integer=2**100000```

In [ ]:
# Run timeit and lprun here

very_long_integer = 2**100000


### Exercise 2 - Summary

Think about what these results are telling you. Some things to highlight:

* You are learning how to identify which parts of a function you need to think about when optimising. Never bother optimising any part of your code that is not taking a significant fraction of the total time.
* The optimal solution to a problem can depend on the input. For shorter inputs, converting an integer to a string and then back to a list of integers adds an overhead that is the dominant computational cost. However, as the input integers become very large, operations like dividing by 10 (which for a computer is not as trivial as it seems, because computers think in binary numbers, not base-10 numbers) become expensive (look up how python handles big integers if you want to understand this better). In this case the overhead of converting to a string is faster.
* Although the second case was faster for normal-sized integers, it doesn't seem that using it would really be a good decision if the code is more difficult to parse.

## Exercise 3 - Identifying common elements in two large lists


This interactive example demonstrates that is often important to use the right tool, or computational method, for the task at hand.

An example of this is the problem I set you last year:

Write a function numpy_nested_sum, which takes as input two 1D numpy arrays (x and y) with lengths N and M respectively and computes the following sum:

$\sum_{i=0}^{N-1} \sum_{j=0}^{M-1} \left(x[i]^2 + y[j]^2 \right)$

Many of you tried something like





In [ ]:
def compute_sum(arrx, arry):
    summ = 0
    for i in range(len(arrx)):
        for j in range(len(arry)):
            summ += arrx[i]**2 + arry[i]**2
    return summ

This directly implements the question as written, but didn't pass the longest test case (due to timeout). Some of you remembered that you are also highly trained mathemeticians, did some algebraic reordering, and realised that this was equal to:

$\sum_{i=0}^{N-1} (M * x[i]^2) +  \sum_{j=0}^{M-1} ( N * y[j]^2)$

This is then coded as


In [ ]:
def compute_sum(arrx, arry):
    summ = 0
    for i in range(len(arrx)):
        summ += M * arrx[i]**2

    for j in range(len(arry)):
        summ += N * arry[i]**2
    return summ

and those students got full marks. The hint in the function name (to call this *nested* sum), was a bit unfair of course. However, the principal here is the important one: How many operations do you actually need to do to get the necessary output? This is called "complexity analysis" in computing, this article has a lot more details on this:

https://www.freecodecamp.org/news/big-o-notation-why-it-matters-and-why-it-doesnt-1674cfa8a23c/


Let's consider a different problem:

* Consider two large lists of integers. Identify integers which are in *both* lists

First let's create the lists and then let's take a first stab at this

In [ ]:
# Let's create our lists first - DO NOT change this code
import numpy
list_a = numpy.random.randint(0,10000000,size=[100000])
list_b = numpy.random.randint(0,10000000,size=[100000])

In [ ]:
# Then let's write our first solution
def identify_common_elements(list_1, list_2):
    common_elements = []
    # Let's just write a for loop, and for every element check if it's in list_2
    for elem in list_1:
        if elem in list_2:
            common_elements.append(elem)
    return common_elements

# This seems like it should be fast, right??

As before, run timeit and lprof on this code to determine run-time and the slowest line. *Before* running lprof identify where you think the most time will be taken, in which line of the code. Were you right?

In [ ]:
# Run timeit and lprun here


Most of the time is spent determining if elem is in list 2, for every element. There are in total `O(N**2)` operations in this operation, where N is the length of the array. Specifically for each of the `N` elements in list 1 we have to check if any of the `N` elements in list 2 are the same. (There would be some special cases here, for example if the lists have a length of 1000000, but contained only integers between 0 and 10).

Can we do better? How can we speed up the `elem in list_2` part of the code? How about this:

In [ ]:
# Let's try that again
def identify_common_elements_with_a_set(list_1, list_2):
    # ALL I DO IS ADD THE FOLLOWING 1 LINE
    list_2 = set(list_2)
    common_elements = []
    # Let's just write a for loop, and for every element check if it's in list_2
    for elem in list_1:
        if elem in list_2:
            common_elements.append(elem)
    return common_elements

# This seems like it wouldn't be any faster, right??

Again use timeit and lprof to profile the code

In [ ]:
# Run timeit and lprun here


Think about why that happened?

Sometimes optimisations in python aren't obvious, because a lot of the internal details can be hidden. A `set` and a `list` are similar but very different objects. On the face of it a `set` just contains a list of non-duplicated entries (So `set([1,1,2,3,4])` = `set([1,2,3,4])`). But because non-duplicate entries are not possible the set can check if an item is in the `set`*much* more quickly than you can check if an item is in a list ... Basically python implements a technique called a "hash table" https://en.wikipedia.org/wiki/Hash_table to decide quickly if an object is already in the set. Dictionaries use the same technique with their keys. (Also look up binary search tables as another interesting technique for this kind of thing).

In [ ]:
# One more try
def identify_common_elements_with_two_sets(list_1, list_2):
    list_1 = set(list_1)
    list_2 = set(list_2)
    return list(list_1 & list_2)


Again profile this code, is it quicker than the previous version?

In [ ]:
# Run timeit and lprun here


You can use the `&` operator to take the combination of the two lists. This results in a code with much fewer lines, but the overall cost is much the same. *Converting* a list to a set would be a non-negligible cost here, but if your input is already a set, and your output can be a set then this operation will be faster.:

In [ ]:
set_a = set(list_a)
set_b = set(list_b)
%timeit set_a & set_b

4.12 ms ± 16.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Can we do even better and record integers that occur multiple times in both lists? This algorithm sorts the input and then walks through the sorted arrays looking for duplicates. The sorting is a `N log N` operation, and the walkthrough requires `O(N)` operations.

In [ ]:
# Let's try that again
def identify_common_elements_with_a_walk(list_1, list_2):
    list_1 = sorted(list_1)
    list_2 = sorted(list_2)
    idx_1 = 0
    idx_2 = 0
    l1 = len(list_1)
    l2 = len(list_2)
    common_elements = []
    curr_1 = list_1[idx_1]
    curr_2 = list_2[idx_2]
    while 1:
        if curr_1 == curr_2:
            common_elements.append(curr_1)
            idx_1 += 1
            if idx_1 == l1:
                break
            curr_1 = list_1[idx_1]
            idx_2 += 1
            if idx_2 == l2:
                break
            curr_2 = list_2[idx_2]
        elif curr_1 < curr_2:
            idx_1 += 1
            if idx_1 == l1:
                break
            curr_1 = list_1[idx_1]
        else:
            idx_2 += 1
            if idx_2 == l2:
                break
            curr_2 = list_2[idx_2]
    return common_elements

In [ ]:
# Run timeit and lprun here


For the data arrays we are considering this is not faster, even if the arrays are sorted before sending to the function. That's probably not surprising. The `&` method on the two sets would have implemented this if it were faster!

This particular example is a good use case for Cython if further optimization is required. More on that in week 11!

## Exercise 4 - Sorting an array

Sorting arrays is a common operation. Making a sorting algorithm that is *fast* is very important and there are many different approaches to the problem (python, for example uses "Timsort"):

https://en.wikipedia.org/wiki/Sorting_algorithm

There is a rule though: If you have an array that contains random input, then you will require an average of `O(N log(N))` operations to sort the array. If the array is *not* random though, this can be shorter (e.g. if the array is already sorted, this can be done in `O(N)`, which is the cost required to identify that the array *is* sorted.

As this is the last exercise, let's make this challenging:

* Write your own code that will take as input an array of numbers. Return a sorted *copy* of the array (the original array should be unchanged). Don't worry too much about making the code fast until you have something that works. When you have something that works, see how you can use the tools presented above to make it as fast as possible.

In [ ]:
import copy
def sort_array(unsorted_array):
    # This line will copy the array so that the input will be preserved.
    # After this you can do what you want to unsorted_array
    unsorted_array = copy.deepcopy(unsorted_array)

    # FIXME: You need to fill this with code DO NOT use python's built-in sort/sorted routines (or any link to
    # that e.g. numpy.sorted). The problem is to think about how you would do this!

    return sorted_array

In [ ]:
# Use this cell to profile your code.

How does your code compare with other code? Here's three examples of pre-existing sorting algorithms. The first is python's built-in `sort` routine:
```
sorted_array = sorted(unsorted_array)
```
The next three functions are implications of
* The "quicksort" technique
* The "mergsort" technique
* The "bubblesort" technique

See here for a description of each of these

https://app.codesignal.com/interview-practice/topics/sorting/tutorial

or here

https://en.wikipedia.org/wiki/Sorting_algorithm

In [ ]:
import copy
def quick_sort(a, l=None, r=None, copied=False):
    if not copied:
        a = copy.deepcopy(a)
    if l is None:
        l = 0
    if r is None:
        r = len(a) - 1
    if l >= r:
        return a

    x = a[l]
    i = l
    j = r

    while i <= j:
        while a[i] < x:
            i += 1
        while a[j] > x:
            j -= 1
        if i <= j:
            t = a[i]
            a[i] = a[j]
            a[j] = t
            i += 1
            j -= 1

    # This is an example of a recursive program. Where we call a function from within the *same* function with a
    # reduced scope. However, this will make profiling hard!
    if (i-1-l) >= 1:
        quick_sort(a, l=l, r=i-1, copied=True)
    if (r-i) >= 1:
        quick_sort(a, l=i, r=r, copied=True)

    return a


def merge_sort(sequence):
    sequence = copy.deepcopy(sequence)

    def merge(sequence, left, middle, right):

        result = []

        i = left
        j = middle
        while i < middle and j < right:
            if sequence[i] < sequence[j]:
                result.append(sequence[i])
                i += 1
            else:
                result.append(sequence[j])
                j += 1

        while i < middle:
            result.append(sequence[i])
            i += 1

        while j < right:
            result.append(sequence[j])
            j += 1

        for i in range(left, right):
            sequence[i] = result[i - left]

    def split(sequence, left, right):
        middle = (left + right) // 2

        if right - left < 2:
            return
        split(sequence, left, middle)
        split(sequence, middle, right)
        merge(sequence, left, middle, right)

    split(sequence, 0, len(sequence))

    return sequence

def bubble_sort(items):
    items = copy.deepcopy(items)

    def swap(firstIndex, secondIndex):
        temp = items[firstIndex]
        items[firstIndex] = items[secondIndex]
        items[secondIndex] = temp

    length = len(items)

    stop = length - 1
    while stop > 0:
        j = 0
        last_swap = 0
        while j < stop:
            if items[j] > items[j + 1]:
                swap(j, j + 1)
                last_swap = j
            j += 1
        stop = last_swap
    return items


For each of the arrays below, time how long it takes to sort the array with each of the methods above, including your own function and the built-in `sorted` function.

In [ ]:
# Array of random numbers
array_a = numpy.random.randint(0,100,size=1000)
# An already sorted array
array_b = numpy.arange(1000)
# An *inversely* sorted array
array_c = numpy.arange(1000)[::-1]
# Two sorted arrays appended together
array_d = numpy.append(numpy.arange(500),numpy.arange(500))

# Time the various sorting algorithms below


## Acknowledgements

With thanks to the following

* https://ipython-books.github.io
* https://jakevdp.github.io/PythonDataScienceHandbook
* https://app.codesignal.com